# Purpose
predict client’s sex through transactions

In [1]:
import pandas as pd
import numpy as np

from sklearn import tree
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model

from xgboost import XGBClassifier

import lightgbm as lgb

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [15]:
filepath = 'C:/Users/nowic/Desktop/home_tasks/pets/sex_classifier/raw_data/'

In [55]:
transactions = pd.read_csv(f'{filepath}to_model.csv')
transactions.drop(['client_id', 'day', 'time', 'code'], axis=1, inplace=True)
transactions

,month,type_description,amount,target
0,8,0,-7928.08,0
1,14,3,-56.15,0
2,5,0,-4689.47,1
3,13,0,-3703.52,1
4,9,1,67377.47,1
...,...,...,...,...
5186,0,2,-22459.16,1
5187,14,1,44918.32,1
5188,7,2,-128017.20,1
5189,8,0,-9334.03,1


In [56]:
X = transactions.drop('target', axis=1)
y = transactions['target']

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, random_state=9)


In [57]:
X_train

,month,type_description,amount
1143,0,3,-13475.49
4009,5,1,-6737.75
3163,2,2,-67377.47
1135,5,0,-2904.64
2526,13,0,-4491.83
...,...,...,...
5014,8,0,-3181.79
1787,13,0,-2357.99
501,13,1,-112295.79
4444,6,2,-4491.83


# RandomForestClassifier

In [58]:
params = {'criterion': ['gini', 'entropy'],
          'max_depth': range(1, 31),
          'min_samples_leaf': range(1, 21),
          'n_estimators': range(1, 11)}

grid_search = model_selection.GridSearchCV(RandomForestClassifier(),
                                           param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 31),
                         'min_samples_leaf': range(1, 21),
                         'n_estimators': range(1, 11)})

In [59]:
grid_search.best_params_

{'criterion': 'gini',
 'max_depth': 3,
 'min_samples_leaf': 18,
 'n_estimators': 3}

In [60]:
estimator = grid_search.best_estimator_
estimator.score(X_test, y_test)

0.5462249614791987

# KNeighborsClassifier

In [61]:
params = {'n_neighbors': range(5, 15),
          'weights': ['uniform', 'distance'],
          'algorithm': ['ball_tree', 'kd_tree', 'brute']}

grid_search = model_selection.GridSearchCV(KNeighborsClassifier(),
                                           param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': range(5, 15),
                         'weights': ['uniform', 'distance']})

In [62]:
grid_search.best_params_

{'algorithm': 'ball_tree', 'n_neighbors': 14, 'weights': 'uniform'}

In [63]:
estimator = grid_search.best_estimator_
estimator.score(X_test, y_test)

0.5100154083204931

# DecisionTreeClassifier

In [64]:
params = {'criterion': ['gini', 'entropy'],
          'max_depth': range(1, 31),
          'min_samples_leaf': range(1, 21)}

grid_search = model_selection.GridSearchCV(tree.DecisionTreeClassifier(),
                                           param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 31),
                         'min_samples_leaf': range(1, 21)})

In [65]:
grid_search.best_params_

{'criterion': 'entropy', 'max_depth': 2, 'min_samples_leaf': 6}

In [66]:
estimator = grid_search.best_estimator_
estimator.score(X_test, y_test)

0.5462249614791987

# XGBClassifier

In [15]:
params = {'n_estimators': range(1, 11),
          'max_depth': range(1, 31),
          'learning_rate': np.arange(0.1, 1, 0.1),
          'verbosity': range(1, 4),
          'booster': ['gbtree', 'gblinear', 'dart']}

grid_search = model_selection.GridSearchCV(XGBClassifier(),
                                           param_grid=params, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

[21:40:32] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,...
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_

In [16]:
grid_search.best_params_

{'booster': 'gbtree',
 'learning_rate': 0.2,
 'max_depth': 2,
 'n_estimators': 3,
 'verbosity': 1}

In [39]:
estimator = grid_search.best_estimator_
estimator.score(X_test, y_test)

0.5485362095531587